# Logger



| ID | English Name of Puzzles | Chinese Translation | Finished? |
| :----:  | :-----: | :-----: | :----: |
| 1 | Standard Sudoku  | 标准数独 | ✅ |
| 2 | Alphadoku | 25 by 25 字母独| ✅  |
| 3 | Killer Sudoku | 杀手数独 | ✅  |
| 4 | Petite Killer Sudoku(X-sum Sudoku)| 小杀手数独 | ✅ |
| 5 | Consecutive Sudoku | 连续数独 | ✅ |
| 6 | Sandwich Sudoku | 三明治数独 | ✅ |
| 7 | Akari (AKA: light UP!) | 照明 | ✅ |
| 8 | Cryptarithmetic Puzzle | 破译密码 | ✅ |
| 9 | Norinori  | 海苔 | ✅ |
| 10 |  thermometer Sudoku | 温度计数独 | ✅ |
| 11 | Number Link | 数链 | ✅ |
| 12 |  Minesweeper | 静态扫雷 | ✅  |
| 13 |  Simple Loop | 简单回路 | ❌  |
| 14 | Siltherlink | 环 | ❌ |
| 15 | Jigsaw Sudoku | 锯齿数独 | ❌ | 
| 16 | Anti-Knight Sudoku | 无马数独 | ✅ | 
| 17 | Anti-King Sudoku | 无缘数独 | ✅ |
| 18 | Black-White Sudoku | 黑白点数独 | ❌ |
| 19 | Arrow Sudoku | 箭头数独 | ✅ |
| 20 | Greater-Than Sudoku | 不等式数独 | ✅ |



# Reference

> https://github.com/dguzelkokar/Solving-Akari-using-Google-OR-Tools
> 
> src: https://www.kaggle.com/code/pintowar/modeling-a-sudoku-solver-with-or-tools
> 
> src: [Jigsaw](https://cn.gridpuzzle.com/jigsaw-killer-sudoku)
> 
> src: [hakank's site](http://www.hakank.org/google_or_tools/)
>
> [Puzzle Site](https://puzzlemadness.co.uk/slitherlink/large/2023/10/18)
> 
> Sandwich Sudoku: [Github Repo](https://github.com/awkwardbunny/z3-sudoku) in z3 solver


In [ ]:
# https://github.com/CPMpy/cpmpy/

# nonogram 一款很有意思的游戏


https://github.com/CPMpy/cpmpy/

nonogram 一款很有意思的游戏

Silther Link Game，有翻译叫“数回”

> https://cs.carleton.edu/cs_comps/1516/slither/index.php


In [3]:
ori = set(["12", "56", "78"])
a = set(["9283", "4378", "7392321"])

ori.remove("12")
print(ori)

{'56', '78'}


In [114]:

from itertools import combinations
from ortools.sat.python import cp_model as cp
import math

class CompoundSudokuSolver:
    
    def __init__(
        self, 
        grid, 
        X = 9, 
        Y = 9, 
        std_rule = True,
        killer = None,
        petite_killer = None,
        consecutive = None,
        sandwich = None,
        anti_knight = None,
        anti_king = None,
        arrow = None,
        thermo = None, 
        greater_than = None,
        diagonal = False) -> None:
        
        self.grid = grid
        self.X = X
        self.Y = Y
        self.std_rule = std_rule
        self.model = cp.CpModel()
        self.solver = cp.CpSolver()
        self.x = {}
        self.killer = killer
        self.petite_killer = petite_killer
        self.consecutive = consecutive
        self.sandwich = sandwich
        self.anti_knight = anti_knight
        self.anti_king = anti_king
        self.arrow = arrow
        self.thermo = thermo
        self.diagonal = diagonal
        self.greater_than = greater_than
        self.aux_sand = None
        
        assert X == Y
        
        for i in range(Y):
            for j in range(X):
                if self.grid[i * self.Y + j] == "0":
                    self.x[i, j] = self.model.NewIntVar(1, self.X, f'x[{i}, {j}]')
                else:
                    self.x[i, j] = int(self.grid[i * self.X + j])
                
    def addStandardConstr(self):
        
        """_summary_
            Add standard Sudoku Constraint
        """
        
        for i in range(self.Y):
            row = [self.x[i, j] for j in range(self.X)]
            self.model.AddAllDifferent(row)
            col = [self.x[j, i] for j in range(self.Y)]
            self.model.AddAllDifferent(col)
        
        for i in range(int(math.sqrt(self.X)) - 1):
            for j in range(int(math.sqrt(self.Y)) - 1):
                l = int(math.sqrt(self.X))

                cell = [
                    self.x[r, c]
                    for r in range(i* l, i * l + l)
                    for c in range(j* l, j * l + l)
                ]
                self.model.AddAllDifferent(cell)
    
    def addConsecutiveConstr(self):
        
        for _, cage in enumerate(self.consecutive):
            self.model.AddAbsEquality(1, self.x[cage[0][0], cage[0][1]] - self.x[cage[1][0], cage[1][1]])
             
    def addDiagonalConstr(self):
        
        """_summary_
            Add diagonal Constraints.
        """
        
        ltr = []
        rtl = []
        for i in range(self.Y):
            ltr.append(self.x[i, i])
            rtl.append(self.X - i - 1, i)
        self.model.AddAllDifferent(ltr)
        self.model.AddAllDifferent(rtl)
            
    def printgrid(self):    
        
        """_summary_
            Print the final grid
        """
    
        for i in range(self.Y):
            for j in range(self.X):
                print(self.solver.Value(self.x[i, j]), end=" ")
            print()
        print()

        print("NumConflicts:", self.solver.NumConflicts())
        print("NumBranches:", self.solver.NumBranches())
        print("WallTime:", self.solver.WallTime())
        
        # for i in range(9):
        #     for j in range(6):
        #         print(self.solver.Value(self.aux_sand["R", i, j]))
    
    def addKillerConstr(self):
        """_summary_
            Add killer constraint from data
        """
        
        for (res, segment) in self.killer:
            cage = [self.x[i[0], i[1]] for i in segment]
            self.model.Add(sum(cage) == res)
            self.model.AddAllDifferent(cage)
    
    def addPetiteKillerConstr(self):
        """_summary_
            增加小杀手数独约束
        Raises:
            Exception: _description_
            Exception: _description_
        """
        if len(self.petite_killer[0]) != self.X:
            raise Exception("小杀手数独斜行和数量有误。")
        for line in self.petite_killer:
            if line[0] == "TL":
                for i in range(self.X - 1):
                    constrCages = []
                    for j in range(i + 1):
                        constrCages.append(self.x[j, i - j])
                    self.model.Add(sum(constrCages) == line[i + 1])

            elif line[0] == "RT":
                
                for i in range(self.X - 1):
                    constrCages = []
                    for j in range(i + 1):
                        constrCages.append(self.x[j, self.X - i + j - 1])
                    self.model.Add(sum(constrCages) == line[i + 1])

            elif line[0] == "BR":
                for i in range(self.X - 1):
                    constrCages = []
                    for j in range(self.X - 1 - i):
                        constrCages.append(self.x[self.X - 1 - j, 1 + i +  j])
                    self.model.Add(sum(constrCages) == line[i + 1])

                    
            elif line[0] == "LB":
            
                for i in range(self.X - 1):
                    constrCages = []
                    for j in range(self.X - 1 - i):
                        constrCages.append(self.x[1 + j + i,  j])
                    self.model.Add(sum(constrCages) == line[i + 1])

            else:
                raise Exception("检查小杀手数独位置元素字符串, 必须为TL / RT / BR / LB 中的一个")

    def addThermoConstr(self):
        """_summary_
            Thermo constraints
        """
        for thermo_ in self.thermo:
            for idx in range(len(thermo_) - 1):
                self.model.Add(self.x[thermo_[idx][0], thermo_[idx][1]] < self.x[thermo_[idx + 1][0], thermo_[idx + 1][1]])
        
    def addSandwichConstr(self):
        """
            Add sandwich Sudoku Constrs
        """
        self.aux_sand = {}
        
        # x[i,j] = 9 -> Bool = True
        # x[i,j] <= 8 -> Bool = False
        # False -> x[i, j] <= 8
        
        # True
        for nums in self.sandwich:
            if nums[0] == "R":
                for idx, num in enumerate(nums[1:]):
                    if num < 0:
                        continue
                    # cnt = 0
                    for index, cbn in enumerate(combinations([i for i in range(self.X)], 2)):

                        self.aux_sand["R", idx, index,  0] = self.model.NewBoolVar(f"R, {idx}, {index}, 0")
                        self.aux_sand["R", idx, index,  1] = self.model.NewBoolVar(f"R, {idx}, {index}, 1")
                        self.aux_sand["R", idx, index,  2] = self.model.NewBoolVar(f"R, {idx}, {index}, 2")
                        self.aux_sand["R", idx, index,  3] = self.model.NewBoolVar(f"R, {idx}, {index}, 3")
                        self.aux_sand["R", idx, index,  4] = self.model.NewBoolVar(f"R, {idx}, {index}, 4")
                        self.aux_sand["R", idx, index,  5] = self.model.NewBoolVar(f"R, {idx}, {index}, 5")

                        
                        self.model.Add(self.x[cbn[0], idx] == 1).OnlyEnforceIf(self.aux_sand["R", idx, index, 0])
                        self.model.Add(self.x[cbn[0], idx] >= 2).OnlyEnforceIf(self.aux_sand["R", idx, index, 0].Not())

                        
                        self.model.Add(self.x[cbn[1], idx] == 9).OnlyEnforceIf(self.aux_sand["R", idx, index, 1])
                        self.model.Add(self.x[cbn[1], idx] <= 8).OnlyEnforceIf(self.aux_sand["R", idx, index, 1].Not())
                        
                        self.model.Add(self.aux_sand["R", idx,  index, 0] + self.aux_sand["R", idx, index,  1] == 2).OnlyEnforceIf(self.aux_sand["R", idx, index, 4])
                        self.model.Add(self.aux_sand["R", idx,  index, 0] + self.aux_sand["R", idx, index,  1] <= 1).OnlyEnforceIf(self.aux_sand["R", idx, index, 4].Not())
                        
                        self.model.Add(sum(self.x[j, idx] for j in range(cbn[0] + 1, cbn[1] )) == num).OnlyEnforceIf(self.aux_sand["R",idx, index, 4])
                        # self.model.Add(sum(self.x[j, idx] for j in range(cbn[0] + 1, cbn[1] - 1)) != num).OnlyEnforceIf(self.aux_sand["R",idx, index, 4])
                        
                        self.model.Add(self.x[cbn[0], idx] == 9).OnlyEnforceIf(self.aux_sand["R", idx, index,  2])
                        self.model.Add(self.x[cbn[0], idx] <= 8).OnlyEnforceIf(self.aux_sand["R", idx, index,  2].Not())

                        
                        self.model.Add(self.x[cbn[1], idx] == 1).OnlyEnforceIf(self.aux_sand["R", idx, index,  3])
                        self.model.Add(self.x[cbn[1], idx] >= 2).OnlyEnforceIf(self.aux_sand["R", idx, index,  3].Not())

                        
                        self.model.Add(self.aux_sand["R", idx, index, 2] + self.aux_sand["R", idx, index,  3] == 2).OnlyEnforceIf(self.aux_sand["R", idx, index, 5])
                        self.model.Add(self.aux_sand["R", idx, index, 2] + self.aux_sand["R", idx, index,  3] <= 1).OnlyEnforceIf(self.aux_sand["R", idx, index, 5].Not())
                        
                        self.model.Add(sum(self.x[j, idx] for j in range(cbn[0] + 1, cbn[1] )) == num).OnlyEnforceIf(self.aux_sand["R",idx, index, 5])
                       
                   
                        
            elif nums[0] == "C":
                if nums[0] == "R":
                    for idx, num in enumerate(nums[1:]):
                        if num < 0:
                            continue
                        # cnt = 0
                        for index, cbn in enumerate(combinations([i for i in range(self.X)], 2)):

                            self.aux_sand["C", idx, index,  0] = self.model.NewBoolVar(f"R, {idx}, {index}, 0")
                            self.aux_sand["C", idx, index,  1] = self.model.NewBoolVar(f"R, {idx}, {index}, 1")
                            self.aux_sand["C", idx, index,  2] = self.model.NewBoolVar(f"R, {idx}, {index}, 2")
                            self.aux_sand["C", idx, index,  3] = self.model.NewBoolVar(f"R, {idx}, {index}, 3")
                            self.aux_sand["C", idx, index,  4] = self.model.NewBoolVar(f"R, {idx}, {index}, 4")
                            self.aux_sand["C", idx, index,  5] = self.model.NewBoolVar(f"R, {idx}, {index}, 5")

                            
                            self.model.Add(self.x[idx, cbn[0]] == 1).OnlyEnforceIf(self.aux_sand["C", idx, index, 0])
                            self.model.Add(self.x[idx, cbn[0]] >= 2).OnlyEnforceIf(self.aux_sand["C", idx, index, 0].Not())

                            
                            self.model.Add(self.x[idx, cbn[1]] == 9).OnlyEnforceIf(self.aux_sand["C", idx, index, 1])
                            self.model.Add(self.x[idx, cbn[1]] <= 8).OnlyEnforceIf(self.aux_sand["C", idx, index, 1].Not())
                            
                            self.model.Add(self.aux_sand["C", idx,  index, 0] + self.aux_sand["C", idx, index,  1] == 2).OnlyEnforceIf(self.aux_sand["C", idx, index, 4])
                            self.model.Add(self.aux_sand["C", idx,  index, 0] + self.aux_sand["C", idx, index,  1] <= 1).OnlyEnforceIf(self.aux_sand["C", idx, index, 4].Not())
                            
                            self.model.Add(sum(self.x[idx, j] for j in range(cbn[0] + 1, cbn[1] )) == num).OnlyEnforceIf(self.aux_sand["R",idx, index, 4])
                            # self.model.Add(sum(self.x[j, idx] for j in range(cbn[0] + 1, cbn[1] - 1)) != num).OnlyEnforceIf(self.aux_sand["R",idx, index, 4])
                            
                            self.model.Add(self.x[idx, cbn[0]] == 9).OnlyEnforceIf(self.aux_sand["C", idx, index,  2])
                            self.model.Add(self.x[idx, cbn[0]] <= 8).OnlyEnforceIf(self.aux_sand["C", idx, index,  2].Not())

                            
                            self.model.Add(self.x[idx, cbn[1]] == 1).OnlyEnforceIf(self.aux_sand["C", idx, index,  3])
                            self.model.Add(self.x[idx, cbn[1]] >= 2).OnlyEnforceIf(self.aux_sand["C", idx, index,  3].Not())

                            
                            self.model.Add(self.aux_sand["C", idx, index, 2] + self.aux_sand["C", idx, index,  3] == 2).OnlyEnforceIf(self.aux_sand["C", idx, index, 5])
                            self.model.Add(self.aux_sand["C", idx, index, 2] + self.aux_sand["C", idx, index,  3] <= 1).OnlyEnforceIf(self.aux_sand["C", idx, index, 5].Not())
                            self.model.Add(sum(self.x[idx, j] for j in range(cbn[0] + 1, cbn[1] )) == num).OnlyEnforceIf(self.aux_sand["R",idx, index, 4]) 

    def addAntiKnightConstr(self):
        
        offsets = [
            (-2, 1), 
            (-2, -1), 
            (-1, 2), 
            (-1, -2), 
            (1, 2), 
            (1, -2), 
            (2, 1), 
            (2, -1)
        ]
        for i in range(self.Y):
            for j in range(self.X):
                for offset in offsets:
                    if (i + offset[0] >= 0 and i + offset[0] < self.Y) and (j + offset[1] >= 0 and j + offset[1] < self.X):
                        self.model.Add(self.x[i, j] != self.x[i + offset[0], j + offset[1]])
        
    def addAntiKingConstr(self):
        
        offs = [-1, 0, 1]
        for i in range(self.Y):
            for j in range(self.X):
                for ofx in offs:
                    for ofy in offs:
                        if (i + ofx >= 0 and i + ofx < self.Y) and (j + ofy >= 0 and j + ofy < self.X) and (ofy != 0 and ofx != 0):
                            self.model.Add(self.x[i, j] != self.x[i + ofx, j + ofy])
        
    def addArrowConstr(self):

        for arrow_ in self.arrow:
            self.model.Add(self.x[arrow_[0][0], arrow_[0][1]] == sum([self.x[ar[0], ar[1]] for ar in arrow_[1:]]))
            
    def addGreaterThanConstr(self):
        for idx, oper in enumerate(self.greater_than):
            cage, sub_cage =  (idx ) // 12 , (idx ) % 12
            if oper == "-":
                continue
            central_x, central_y =   (cage % 3) * 3 + 1, (cage // 3) * 3  + 1
            if sub_cage == 0:
                self.model.Add(self.x[central_y - 1, central_x - 1] > self.x[ central_y - 1, central_x]) if oper == ">" else  self.model.Add(self.x[central_y - 1, central_x - 1] < self.x[central_y - 1, central_x])
                
            elif sub_cage == 1:
                self.model.Add(self.x[central_y - 1, central_x] > self.x[central_y - 1, central_x + 1]) if oper == ">" else  self.model.Add(self.x[central_y - 1, central_x] < self.x[central_y - 1, central_x + 1])
                
            elif sub_cage == 2:
                self.model.Add(self.x[central_y - 1, central_x - 1] < self.x[central_y, central_x - 1]) if oper == ">" else  self.model.Add(self.x[central_y - 1, central_x - 1 ] > self.x[central_y, central_x - 1])
            elif sub_cage == 3:
                self.model.Add(self.x[central_y - 1, central_x  ] < self.x[ central_y, central_x ]) if oper == ">" else  self.model.Add(self.x[central_y - 1, central_x ] > self.x[central_y, central_x])
            elif sub_cage == 4:
                self.model.Add(self.x[central_y - 1, central_x + 1 ] < self.x[central_y, central_x + 1 ]) if oper == ">" else  self.model.Add(self.x[central_y - 1 , central_x + 1] > self.x[central_y, central_x + 1])
            elif sub_cage == 5:
                self.model.Add(self.x[central_y, central_x - 1] > self.x[central_y, central_x]) if oper == ">" else  self.model.Add(self.x[central_y, central_x - 1] < self.x[central_y, central_x])
            elif sub_cage == 6:
                self.model.Add(self.x[central_y, central_x] > self.x[central_y, central_x + 1]) if oper == ">" else  self.model.Add(self.x[central_y, central_x] < self.x[central_y, central_x + 1])
            elif sub_cage == 7:
                self.model.Add(self.x[central_y, central_x  - 1  ] < self.x[central_y + 1, central_x  - 1  ]) if oper == ">" else  self.model.Add(self.x[central_y , central_x  - 1 ] > self.x[central_y + 1, central_x  - 1 ])
            elif sub_cage == 8:
                self.model.Add(self.x[central_y, central_x ] < self.x[central_y + 1, central_x ]) if oper == ">" else  self.model.Add(self.x[central_y, central_x ] > self.x[central_y + 1, central_x])
            elif sub_cage == 9:
                self.model.Add(self.x[central_y, central_x  + 1 ] < self.x[central_y + 1, central_x + 1 ]) if oper == ">" else  self.model.Add(self.x[central_y, central_x + 1 ] > self.x[central_y + 1, central_x + 1])
            elif sub_cage == 10:
                self.model.Add(self.x[central_y + 1, central_x - 1] > self.x[central_y + 1, central_x]) if oper == ">" else  self.model.Add(self.x[central_y + 1, central_x - 1] < self.x[central_y + 1, central_x])
            elif sub_cage == 11:
                self.model.Add(self.x[central_y + 1, central_x] > self.x[central_y + 1, central_x + 1]) if oper == ">" else  self.model.Add(self.x[central_y + 1, central_x] < self.x[central_y + 1, central_x + 1])
    

    def solveall(self):

        if self.std_rule:
            self.addStandardConstr()
        if self.diagonal:
            self.addDiagonalConstr()
        if self.killer != None:
            self.addKillerConstr()
        if self.petite_killer != None:
            self.addPetiteKillerConstr()
        if self.consecutive != None:
            self.addConsecutiveConstr()
        if self.sandwich != None:
            self.addSandwichConstr()
        if self.anti_knight != None and self.anti_knight == True:
            self.addAntiKnightConstr()
        if self.anti_king != None and self.anti_king == True:
            self.addAntiKingConstr()
        if self.thermo != None:
            self.addThermoConstr()
        if self.arrow != None:
            self.addArrowConstr()
        if self.greater_than != None:
            if len(self.greater_than) == 108:
                self.addGreaterThanConstr()
            
        
        status = self.solver.Solve(self.model)

        if status == cp.OPTIMAL:
            self.printgrid()
            # for k in range(36):
            #     if self.solver.Value(self.aux_sand["R", 1, k, 0]) == 1 and self.solver.Value(self.aux_sand["R", 1, k , 1]) == 1:
            #         print("FOUND")

            #     if self.solver.Value(self.aux_sand["R", 2, k, 2]) == 1 and self.solver.Value(self.aux_sand["R", 2, k , 3]) == 1:
            #         print("FOUND COL 3")
                
            #     if self.solver.Value(self.aux_sand["R", 2, k, 5]):
            #         print("SSSSSS")

            # print(self.solver.Value(self.aux_sand["R", 1, 3, 0]))
            # print(self.solver.Value(self.aux_sand["R", 1, 3, 1]))
            # print(self.solver.Value(self.aux_sand["R", 2, 13, 2]))
            # print(self.solver.Value(self.aux_sand["R", 2, 13, 3]))
            # print(self.solver.Value(self.aux_sand["R", 2, 13, 5]))
        elif status == cp.INFEASIBLE:
            print("模型不可行")
            # self.model.ExportToFile("./help.txt")
        else:
            print("无法在规定时间内求解")
        # else:
        #     print("没有找到可行方案")
        #     self.model.ExportToFile("./help.txt")




In [115]:
if __name__ == "__main__":
    test_greater_than_solver = "000000000000000000000000000000000000000000000000000000000000000000000000000000000"
    css = CompoundSudokuSolver(grid = test_greater_than_solver, greater_than = "<-<->--<<>-->----><---<>>--><>---<---><->>>>--<<-<-<>--<>->----->---><<-<--<<-->----->--<-->><--><----->---<")
    css.solveall()

#  If the number below < the number above, the sign is "<", 
# If the number below > the number above, the sign is ">",
# For each cell, the sign is sequenced from left to right and from top to bottom.
# and iterate cell from left to right and from top to bottom.

# 如何表示这个盘，就是每次遍历一个宫，每个宫最多有12个符号。我们按照从左到右从上到下遍历9个宫，
# 并且按照从上到下从左到右遍历每个宫的12个符号，连在一起就构成了108个符号，表示出整个棋盘。

3 5 4 7 6 9 2 1 8 
2 9 6 5 1 8 4 3 7 
1 7 8 3 4 2 9 5 6 
7 2 1 9 5 6 3 8 4 
5 4 3 8 2 7 1 6 9 
6 8 9 4 3 1 5 7 2 
4 6 7 1 9 5 8 2 3 
8 3 5 2 7 4 6 9 1 
9 1 2 6 8 3 7 4 5 

NumConflicts: 0
NumBranches: 0
WallTime: 0.019632


![](./assets/figures/GreaterThan1.png)

![](./assets/figures/GreaterThan2.png)

In [97]:
css.model.ExportToFile("./help.txt")

True

In [15]:
# 测试箭头数独
 

if __name__ == "__main__":
    test_arrow_sudoku_grid = "200000005000100000005020400060003090000040000080500070007060900000007000800000007"
    test_arrow_sudoku_arrow = [
        [(0,4), (0,2), (1,1), (2,0)],
        [(4,8),(2,8),(1,7),(0,6)],
        [(4,0),(6,0),(7,1),(8,2)],
        [(8,4),(8,6),(7,7),(6,8)]
    ]
    css = CompoundSudokuSolver(grid = test_arrow_sudoku_grid, arrow = test_arrow_sudoku_arrow)
    css.solveall()

2 9 3 7 8 4 1 6 5 
6 4 8 1 3 5 7 2 9 
1 7 5 6 2 9 4 8 3 
5 6 4 2 7 3 8 9 1 
7 1 2 9 4 8 5 3 6 
3 8 9 5 1 6 2 7 4 
4 3 7 8 6 1 9 5 2 
9 2 6 4 5 7 3 1 8 
8 5 1 3 9 2 6 4 7 

NumConflicts: 0
NumBranches: 0
WallTime: 0.027005


# 箭头数独（Arrow Sudoku）

- 标准数独规则适用
- 规定每个Arrow起点的数字等于箭头经过的其他数字之和，注意箭头经过的数字可以是重复的

-----------

- STD rules applied
- The number of origin of arrow = sum of rest numbers in the same arrow.

![](./assets/figures/Arrow%20Sudoku1.png)

![](./assets/figures/Arrow%20Sudoku2.png)


-------


- [Ref:Bilibili article](https://www.bilibili.com/read/cv10214658)

In [100]:
if __name__ == "__main__":
    
    # Test AntiKnight Sudoku
    test_antiknight_grid = "580400027000970000000005030005030000000000680060000300106000000097050000008000000"
    css = CompoundSudokuSolver(grid = test_antiknight_grid, anti_knight=True)
    css.solveall()
    
    # Test Anti-King Sudoku
    test_antiking_grid = "001003070050007009600081000000000745009000300543000000000940003100700050060300200"
    css = CompoundSudokuSolver(grid = test_antiking_grid, anti_king=True)
    css.solveall()

5 8 9 4 6 3 1 2 7 
4 2 3 9 7 1 5 6 8 
6 7 1 8 2 5 4 3 9 
8 1 5 6 3 4 7 9 2 
9 3 2 5 1 7 6 8 4 
7 6 4 2 8 9 3 5 1 
1 4 6 3 9 8 2 7 5 
3 9 7 1 5 2 8 4 6 
2 5 8 7 4 6 9 1 3 

NumConflicts: 0
NumBranches: 0
WallTime: 0.012209000000000001
4 2 1 5 9 3 6 7 8 
3 5 8 6 2 7 4 1 9 
6 9 7 4 8 1 5 3 2 
2 8 6 1 3 9 7 4 5 
7 1 9 8 5 4 3 2 6 
5 4 3 2 7 6 8 9 1 
8 7 5 9 4 2 1 6 3 
1 3 2 7 6 8 9 5 4 
9 6 4 3 1 5 2 8 7 

NumConflicts: 0
NumBranches: 0
WallTime: 0.005952000000000001


In [88]:
# 测试三明治数独

if __name__ == "__main__":

    test_sandwich_grid = "000090064000200100000104007006000000090000806000500000053682010920407005070010020"
    test_sandwich_nums = [
        ["R", 6, 13, 20, 30, 35, 3, 2, 19, 0], 
        ["C", 5,0 ,0, 0, 0, 7, 0, 2, 0]
    ]
    
    css = CompoundSudokuSolver(grid = test_sandwich_grid, sandwich = test_sandwich_nums)
    css.solveall()


7 1 2 3 9 8 5 6 4 
3 4 5 2 7 6 1 9 8 
8 6 9 1 5 4 2 3 7 
5 3 6 8 4 1 9 7 2 
1 9 4 7 2 3 8 5 6 
2 8 7 5 6 9 3 4 1 
4 5 3 6 8 2 7 1 9 
9 2 1 4 3 7 6 8 5 
6 7 8 9 1 5 4 2 3 

NumConflicts: 0
NumBranches: 0
WallTime: 0.017058


# 无马数独(Anti Knight Sudoku)

- 普通数独规则适用
- 国际象棋中马能走到的格子和本格子的数字不能相同

------------------

![](./assets/figures/AntiKnight.png)

--------

Test Case:

![](./assets/figures/Antiknight2.png)

![](./assets/figures/Antiknight3.png)

# 无缘数独(Anti King Sudoku)

- 普通数独规则适用 
- 国际象棋中King一步能走到的位置数字都不相同(也就是左上、上、右上、左、右、左下、下、右下)都不等于该格。

![](./assets/figures/AntiKing.png)


参考B站大佬的讲解和介绍：[传送门](https://www.bilibili.com/read/cv10214643)

# 三明治数独

- 普通数独规则适用
- 行列上的数字表示这一行/列在1和9之间的数字的和


-----------

- Standard Sudoku rules applied.
- Number of row/ col repressent the sum of numbers between 1 and 9 of this col / row.

![](./assets/figures/Sandwich%20Sudoku.png)

![](./assets/figures/Sandwich%20Sudoku1.png)



In [24]:
# 温度计数独

if __name__ == "__main__":
    test_thermo_grid = "785000200000000010000000000000000000000030000000000000000090000004070000100000607"

    test_thermo_cage = [
        [ (0,4), (1,3), (2,2), (3,1) ], 
        [ (4, 8), (3,7), (2,6), (1,5)],
        [(4,5), (3,5), (3,4), (3,3) ], 
        [(4, 3), (5,3), (5,4), (5,5)],
        [ (4,0), (5,1), (6,2), (7,3)],
        [(8,4), (7,5), (6,6), (5,7)]
    ]
    css = CompoundSudokuSolver(grid = test_thermo_grid, thermo = test_thermo_cage)
    css.solveall()
    # Check Completed

7 8 5 3 1 4 2 9 6 
9 4 6 2 5 7 3 1 8 
2 1 3 9 6 8 4 7 5 
6 7 1 5 4 2 8 3 9 
4 9 8 6 3 1 7 5 2 
3 5 2 7 8 9 1 6 4 
8 2 7 1 9 6 5 4 3 
5 6 4 8 7 3 9 2 1 
1 3 9 4 2 5 6 8 7 

NumConflicts: 0
NumBranches: 0
WallTime: 0.011980000000000001


In [13]:
# 标准数独

if __name__ == "__main__":
    test_std_solver = "549001738367008001200073040000900005000705460135840070004000307780350006023080000" 
    css = CompoundSudokuSolver(grid=test_std_solver, X = 9, Y = 9, std_rule = True)
    css.solveall()

5 4 9 2 6 1 7 3 8 
3 6 7 4 9 8 5 2 1 
2 1 8 5 7 3 6 4 9 
4 7 6 9 3 2 8 1 5 
8 9 2 7 1 5 4 6 3 
1 3 5 8 4 6 9 7 2 
6 5 4 1 2 9 3 8 7 
7 8 1 3 5 4 2 9 6 
9 2 3 6 8 7 1 5 4 

NumConflicts: 0
NumBranches: 0
WallTime: 0.003268


In [12]:
if __name__ == "__main__":
    test_greater_than_solver = "000000000000000000000000000000000000000000000000000000000000000000000000000000000"
    css = CompoundSudokuSolver(grid = test_greater_than_solver, greater_than = "<-<->--<<>>------><---<>>--><>---<---><->>>>--<<-<-<>--<>->----->---><<-<--<<-->----->--<-->><--><----->---<")
    css.solveall()


1 3 7 9 8 5 6 4 2 
4 2 5 6 1 3 9 8 7 
6 9 8 2 7 4 1 5 3 
2 5 1 4 9 8 7 3 6 
8 7 3 5 6 2 4 1 9 
9 4 6 7 3 1 5 2 8 
5 1 9 3 2 7 8 6 4 
7 8 2 1 4 6 3 9 5 
3 6 4 8 5 9 2 7 1 

NumConflicts: 0
NumBranches: 1300
WallTime: 0.051246


In [24]:
# 测试杀手数独
if __name__ == "__main__":
    test_killer_solver = "0" * 81
    test_killer_cages = [
        [3, [[0, 0], [0, 1]]],
        [15, [[0, 2], [0, 3], [0, 4]]],
        [22, [[0, 5], [1, 4], [1, 5], [2, 4]]],
        [4, [[0, 6], [1, 6]]],
        [16, [[0, 7], [1, 7]]],
        [15, [[0, 8], [1, 8], [2, 8], [3, 8]]],
        [25, [[1, 0], [1, 1], [2, 0], [2, 1]]],
        [17, [[1, 2], [1, 3]]],
        [9, [[2, 2], [2, 3], [3, 3]]],
        [8, [[2, 5], [3, 5], [4, 5]]],
        [20, [[2, 6], [2, 7], [3, 6]]],
        [6, [[3, 0], [4, 0]]],
        [14, [[3, 1], [3, 2]]],
        [17, [[3, 4], [4, 4], [5, 4]]],
        [17, [[3, 7], [4, 6], [4, 7]]],
        [13, [[4, 1], [4, 2], [5, 1]]],
        [20, [[4, 3], [5, 3], [6, 3]]],
        [12, [[4, 8], [5, 8]]],
        [27, [[5, 0], [6, 0], [7, 0], [8, 0]]],
        [6, [[5, 2], [6, 1], [6, 2]]],
        [20, [[5, 5], [6, 5], [6, 6]]],
        [6, [[5, 6], [5, 7]]],
        [10, [[6, 4], [7, 3], [7, 4], [8, 3]]],
        [14, [[6, 7], [6, 8], [7, 7], [7, 8]]],
        [8, [[7, 1], [8, 1]]],
        [16, [[7, 2], [8, 2]]],
        [15, [[7, 5], [7, 6]]],
        [13, [[8, 4], [8, 5], [8, 6]]],
        [17, [[8, 7], [8, 8]]]
    ]
    
    css = CompoundSudokuSolver(test_killer_solver, killer = test_killer_cages)
    css.solveall()

2 1 5 6 4 7 3 9 8 
3 6 8 9 5 2 1 7 4 
7 9 4 3 8 1 6 5 2 
5 8 6 2 7 4 9 3 1 
1 4 2 5 9 3 8 6 7 
9 7 3 8 1 6 4 2 5 
8 2 1 7 3 9 5 4 6 
6 5 9 4 2 8 7 1 3 
4 3 7 1 6 5 2 8 9 

NumConflicts: 0
NumBranches: 0
WallTime: 0.010778000000000001


# 解谜游戏：杀手数独（Killer Sudoku）


- 标准数独规则适用
- 除了标准数独规则外，限制每个虚线框内的数字之和等于标记的数字。


-------

English Version:

Killer Sudoku, Rules:

- Standard Sudoku Applied.
- Sum of numbers in cage equals the marked number.

![Killer Sudoku](./assets/figures/KillerSudoku.png)

In [19]:
# 测试连续数独

if __name__ == "__main__":
    test_consecutive = "600005000100000000000700000000000800000000000500100000000000000900000000000000000"
    test_consecutive_cage = [
        [(0, 8),  (1, 8) ],
        [(1, 6),  (1, 7) ],
        [(1, 6),  (2, 6) ],
        [(2, 6),  (2, 7) ],
        [(2, 3),  (3, 3) ],
        [(2, 4),  (3, 4) ],
        [(3, 0),  (4, 0) ],
        [(3, 3),  (4, 3) ],
        [(3, 7),  (4, 7) ],
        [(3, 8),  (4, 8) ],
        [(4, 2),  (4, 3) ],
        [(4, 4),  (5, 4) ],
        [(4, 6),  (4, 7) ],
        [(5, 0),  (5, 1) ],
        [(5, 1),  (6, 1) ],
        [(5, 2),  (5, 3) ],
        [(5, 3),  (6, 3) ],
        [(5, 4),  (6, 4) ],
        [(5, 7),  (6, 7) ],
        [(5, 8),  (6, 8) ],
        [(6, 1),  (6, 2) ],
        [(6, 3),  (7, 3) ],
        [(6, 6),  (6, 7) ],
        [(7, 0),  (7, 1) ],
        [(7, 1),  (7, 2) ],
        [(7, 3),  (7, 4) ],
        [(7, 4),  (8, 4) ],
        [(7, 7),  (8, 7) ],
        [(7, 8),  (8, 8) ],
        [(8, 7),  (8, 8) ],
    ]

    css = CompoundSudokuSolver(grid = test_consecutive, std_rule = True, consecutive = test_consecutive_cage)
    css.solveall()

6 2 7 6 4 5 8 3 1 
1 4 5 1 9 8 6 7 2 
6 8 1 7 7 2 5 4 9 
1 7 3 4 8 6 8 9 5 
2 9 4 5 1 3 7 8 6 
5 6 8 1 2 7 4 1 3 
7 5 6 8 3 9 1 2 4 
9 3 2 7 6 1 9 5 8 
8 1 9 2 5 4 3 6 7 

NumConflicts: 395
NumBranches: 3136
WallTime: 0.08524200000000001


# 解决思维谜题：连续数独（Consecutive Sudoku）

- 标准数独规则适用；
- 有标记的相邻的两个格子数字必须是连续的（差的绝对值是1，因此在实现的时候用 `AllAbsEquality` 函数）

------

English Version:

**Solve Consecutive Sudoku**

- Standard Sudoku Rules applied.
- Two adjacent cage with marker bust be integer-consecutive, (e.g. 1,2 or 2,3; 4,5 or 3,4)


-----------

![](./assets/figures/ConsecutiveSudoku.png)

![](./assets/figures/ConsecutiveSudoku1.png)


> 这个题竟然不是唯一解～～

In [20]:
# 测试小杀手数独

if __name__ == "__main__":
    test_petite_killer = "005000000000000000000002000000000000000006040000000000000060000000300000000000008"
    test_petite_killer_cages = [ 
        [ "TL", 2, 10, 19, 23, 19, 25, 45, 47], 
        [ "RT", 4, 13, 16, 13, 20, 30, 45, 47],
        [ "BR", 37, 35, 30, 19, 20, 16, 10, 8],
        [ "LB", 33, 34, 33, 19, 19, 15, 14, 4]
    ]

    css = CompoundSudokuSolver(grid = test_petite_killer, petite_killer=test_petite_killer_cages )
    css.solveall()

2 9 5 6 3 1 7 8 4 
1 8 3 7 4 9 2 6 5 
6 7 4 8 5 2 1 9 3 
7 2 6 4 9 3 8 5 1 
3 1 8 5 7 6 9 4 2 
5 4 9 2 1 8 3 7 6 
8 3 7 1 6 5 4 2 9 
9 6 2 3 8 4 5 1 7 
4 5 1 9 2 7 6 3 8 

NumConflicts: 1342
NumBranches: 3951
WallTime: 0.21344200000000002


# 解决思维谜题：小杀手数独（Petite Killer Sudoku）

- 标准数独规则适用；
- 斜对角线上的数字和等于小数字。注意一共有四个方向的斜对角线，分别用"T","B","L","R" 标记“上(Top)”“下(Bottom)”“左(Left)”“右(Right)”，因此组成了“TL”、“RT”、“BR”、“LB” 四个方向来标记数字。


> Numbers with arrows indicate sum of the numbers in each direction.
------


![](./assets/figures/Petite%20Sudoku.png)

![](./assets/figures/Petite%20Sudoku%201.png)

In [ ]:
from ortools.sat.python import cp_model as cp 
def karoku_solver(grid, X, Y):
    
    pass 